In [1]:
import numpy as np
import pylab as plt
import xarray as xr
from pismragis.trajectories import compute_trajectory, compute_perturbation
from joblib import Parallel, delayed
import geopandas as gp
from pathlib import Path
from pismragis.processing import tqdm_joblib
import pandas as pd
from pyDOE import lhs
from scipy.stats.distributions import uniform
from tqdm.auto import tqdm

In [2]:
ds = xr.open_dataset("~/Google Drive/My Drive/data/ITS_LIVE/GRE_G0240_0000.nc")

In [3]:
url = Path("/Users/andy/Google Drive/My Drive/data/GreenlandFluxGatesAschwanden/greenland-flux-gates-jibneighbors.shp")

In [4]:
sigma = 1
VX = np.squeeze(ds["vx"].to_numpy())
VY = np.squeeze(ds["vy"].to_numpy())
VX_e = np.squeeze(ds["vx_err"].to_numpy())
VY_e = np.squeeze(ds["vy_err"].to_numpy())

x = ds["x"].to_numpy()
y = ds["y"].to_numpy()
nx = len(x)
ny = len(y)

VX_min, VX_max = VX - sigma * VX_e, VX + sigma * VX_e
VY_min, VY_max = VY - sigma * VY_e, VY + sigma * VY_e

In [5]:
n_draw_samples = 20
unif_sample = lhs(2, n_draw_samples)

with tqdm_joblib(tqdm(desc="Processing perturbations", total=n_draw_samples, position=0)) as progress_bar:
    all_perturb_glaciers = Parallel(n_jobs=10)(
        delayed(compute_perturbation)(url, VX_min, VX_max, VY_min, VY_max, x, y, 
                                       perturbation=perturb, sample=unif_sample[perturb, :], 
                                       total_time=1_000, dt=1, reverse=True)
                for perturb in range(n_draw_samples))
    del progress_bar

all_perturb_glaciers = pd.concat(all_perturb_glaciers).reset_index(drop=True)
# all_perturb_glaciers.to_file("all_perturb_traj.gpkg")

Processing perturbations:   0%|          | 0/20 [00:00<?, ?it/s]